In [1]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import time
%matplotlib inline

# concat training data & weath data

In [2]:
print ('start data preparation')

start data preparation


In [3]:
d=pd.read_csv('./trajectories(table 5)_training.csv')
d1=pd.read_csv('./trajectories(table_5)_training2.csv')
w=pd.read_csv('./weather (table 7)_training_update.csv')
w1=pd.read_csv('./weather (table 7)_test1.csv')

In [4]:
data=pd.concat([d,d1],ignore_index=True)
weath=pd.concat([w,w1],ignore_index=True)

In [5]:
data['starting_time']=data['starting_time'].astype(np.datetime64)
data['date']=data['starting_time'].dt.date
data['month']=data['starting_time'].dt.month
data['day']=data['starting_time'].dt.day
data['hour']=data['starting_time'].dt.hour
data['time']=data['starting_time'].dt.minute
data['weekday']=data['starting_time'].dt.weekday+1

In [6]:
#drop data of holiday
c=data.loc[(data['month']==10)&(data['day']==10)|
         ((data['month']==9)&(data['day']==29)&(data['hour']>=21))|
         ((data['month']==9)&(data['day']==30)&(data['hour']<3))]
data=data.drop(data.index[c.index])

In [7]:
#oversize data processing
data.loc[data['travel_time']>=500,'travel_time']=500
#data.loc[data['travel_time']>500]

In [8]:
data.to_csv('train_data.csv',index=False)

# let training data become 5 min step

In [9]:
def avgTravelTime(in_file,path,out_suffix):
    file_suffix = '.csv'
    in_file_name = in_file + file_suffix
    out_file_name = out_suffix + file_suffix

    # Step 1: Load trajectories
    fr = open(path + in_file_name, 'r')
    fr.readline()  # skip the header
    traj_data = fr.readlines()
    fr.close()
    print(traj_data[0])

    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '-' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = math.floor(trace_start_time.minute / 5) * 5
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     int(trace_start_time.hour), int(time_window_minute), 0)
        tt = float(each_traj[5]) # travel time

        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

    # Step 3: Calculate average travel time for each route per time window
    fw = open(out_file_name, 'w')
    fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
    for route in travel_times.keys():
        route_time_windows = list(travel_times[route].keys())
        route_time_windows.sort()
        for time_window_start in route_time_windows:
            time_window_end = time_window_start + timedelta(minutes=5)
            tt_set = travel_times[route][time_window_start]
            avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
            out_line = ','.join(['"' + route.split('-')[0] + '"', '"' + route.split('-')[1] + '"',
                                 '"[' + str(time_window_start) + ',' + str(time_window_end) + ')"',
                                 '"' + str(avg_tt) + '"']) + '\n'
            fw.writelines(out_line)
    fw.close()
    return

In [10]:
in_file = 'train_data'
path='./'
out_suffix = '5min_avg_travel_time_training'
avgTravelTime(in_file,path,out_suffix)

B,3,1065642,2016-07-19 00:14:24,105#2016-07-19 00:14:24#9.56;100#2016-07-19 00:14:34#6.75;111#2016-07-19 00:14:41#13.00;103#2016-07-19 00:14:54#7.47;122#2016-07-19 00:15:02#32.85,70.85,2016-07-19,7,19,0,14,2



In [11]:
in_file = 'trajectories(table 5)_test2'
path='./'
out_suffix = '5min_avg_travel_time_testing'
avgTravelTime(in_file,path,out_suffix)

"A","2","1000775","2016-10-25 06:00:15","110#2016-10-25 06:00:15#10.52;123#2016-10-25 06:00:25#5.09;107#2016-10-25 06:00:30#2.93;108#2016-10-25 06:00:33#3.58;120#2016-10-25 06:00:37#0.79;117#2016-10-25 06:00:38#17.89","40.89"



# training data preprocessing

In [12]:
data=pd.read_csv('5min_avg_travel_time_training.csv')

In [13]:
a=[]
for c in data['time_window']:
    c1=datetime.strptime(c[1:20], "%Y-%m-%d %H:%M:%S")
    a.append(c1)
data['date']=a

In [14]:
data['month']=data['date'].dt.month
data['day']=data['date'].dt.day
data['hour']=data['date'].dt.hour
data['time']=data['date'].dt.minute
data['weekday']=data['date'].dt.weekday+1

In [15]:
data.insert(3, "t1", 0)
data.insert(4, "t2", 0)
data.insert(5, "t3", 0)
data.insert(6, "deltat1", 0)
data.insert(7, "deltat2", 0)
data.insert(15,"pressure", 0)
data.insert(16,"sea_pressure", 0)
data.insert(17,"wind_direction", 0)
data.insert(18,"wind_speed", 0)
data.insert(19,"temperature", 0)
data.insert(20,"rel_humidity", 0)
data.insert(21,"precipitation", 0)

In [16]:
A2=data.loc[(data['intersection_id']=='A')&(data['tollgate_id']==2)]
A3=data.loc[(data['intersection_id']=='A')&(data['tollgate_id']==3)]
B1=data.loc[(data['intersection_id']=='B')&(data['tollgate_id']==1)]
B3=data.loc[(data['intersection_id']=='B')&(data['tollgate_id']==3)]
C1=data.loc[(data['intersection_id']=='C')&(data['tollgate_id']==1)]
C3=data.loc[(data['intersection_id']=='C')&(data['tollgate_id']==3)]

In [17]:
d=[A2,A3,B1,B3,C1,C3]

# missing value imputation

In [18]:
time1=timedelta(days=0,minutes=5)
time2=timedelta(days=0,minutes=10)
time3=timedelta(days=0,minutes=15)

In [19]:
for data in d:
    for c in (data.index):
        data.loc[(data["date"])==(data["date"][c]+time1),'t1']=data["avg_travel_time"][c]
        data.loc[(data["date"])==(data["date"][c]+time2),'t2']=data["avg_travel_time"][c]
        data.loc[(data["date"])==(data["date"][c]+time3),'t3']=data["avg_travel_time"][c]

In [20]:
for data in d:
    for c in range(7):
        for c1 in range(24):
            data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t1']==0),'t1']=data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t1']!=0)]['t1'].mean()
            data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t2']==0),'t2']=data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t2']!=0)]['t2'].mean()
            data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t3']==0),'t3']=data.loc[(data['weekday']==c) & (data['hour']==c1) & (data['t3']!=0)]['t3'].mean()
    for c in range(7):
        for c1 in range(24):
            data.loc[(data['t1'].isnull())&(data['hour']==c1),'t1']=data.loc[(data['hour']==c1)&(data['weekday']!=c)]['t1'].mean()
            data.loc[(data['t2'].isnull())&(data['hour']==c1),'t2']=data.loc[(data['hour']==c1)&(data['weekday']!=c)]['t2'].mean()
            data.loc[(data['t3'].isnull())&(data['hour']==c1),'t3']=data.loc[(data['hour']==c1)&(data['weekday']!=c)]['t3'].mean()
    for c in range(7):
        for c1 in range(24):
            data.loc[(data['t1'].isnull())&(data['hour']==c1) & (data['weekday']==c),'t1']=data.loc[(data['hour']==c1)&(data['weekday']==c)]['avg_travel_time'].mean()
            data.loc[(data['t2'].isnull())&(data['hour']==c1) & (data['weekday']==c),'t2']=data.loc[(data['hour']==c1)&(data['weekday']==c)]['avg_travel_time'].mean()
            data.loc[(data['t3'].isnull())&(data['hour']==c1) & (data['weekday']==c),'t3']=data.loc[(data['hour']==c1)&(data['weekday']==c)]['avg_travel_time'].mean()    

In [21]:
data=pd.concat(d,ignore_index=True)

In [22]:
data['deltat1']=data['t1']-data['t2']
data['deltat2']=data['t2']-data['t3']

# weather data processiong

In [23]:
weath['date']=weath['date'].astype(np.datetime64)
weath['month']=weath['date'].dt.month
weath['day']=weath['date'].dt.day

In [24]:
for c in range(len(weath)):
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'pressure']=weath['pressure'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'sea_pressure']=weath['sea_pressure'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'wind_direction']=weath['wind_direction'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'wind_speed']=weath['wind_speed'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'temperature']=weath['temperature'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'rel_humidity']=weath['rel_humidity'][c]
    data.loc[((data['month']==weath['month'][c])&(data['day']==weath['day'][c]))&
         (((data['hour']-weath['hour'][c])>=0)&((data['hour']-weath['hour'][c])<3)),'precipitation']=weath['precipitation'][c]

In [25]:
a=[]
for c in data['time_window']:
    c1=datetime.strptime(c[1:20], "%Y-%m-%d %H:%M:%S")
    a.append((int(c1.hour)*60+int(c1.minute))/5)
data['check']=a

In [26]:
data.to_csv('training_data.csv',index=False)

# testing data preprocessing

In [27]:
data2=pd.read_csv('5min_avg_travel_time_testing.csv')

In [28]:
a=[]
b=[]
for c in data2['time_window']:
    c1=datetime.strptime(c[1:20], "%Y-%m-%d %H:%M:%S")
    a.append(c1)
data2['date']=a

In [29]:
data2['month']=data2['date'].dt.month
data2['day']=data2['date'].dt.day
data2['hour']=data2['date'].dt.hour
data2['time']=data2['date'].dt.minute
data2['weekday']=data2['date'].dt.weekday+1

In [30]:
data2.insert(3, "t1", 0)
data2.insert(4, "t2", 0)
data2.insert(5, "t3", 0)
data2.insert(6, "deltat1", 0)
data2.insert(7, "deltat2", 0)

In [31]:
A2test=data2.loc[(data2['intersection_id']=='A')&(data2['tollgate_id']==2)]
A3test=data2.loc[(data2['intersection_id']=='A')&(data2['tollgate_id']==3)]
B1test=data2.loc[(data2['intersection_id']=='B')&(data2['tollgate_id']==1)]
B3test=data2.loc[(data2['intersection_id']=='B')&(data2['tollgate_id']==3)]
C1test=data2.loc[(data2['intersection_id']=='C')&(data2['tollgate_id']==1)]
C3test=data2.loc[(data2['intersection_id']=='C')&(data2['tollgate_id']==3)]

In [32]:
d=[A2,A3,B1,B3,C1,C3]

In [33]:
d1=[A2test,A3test,B1test,B3test,C1test,C3test]

In [34]:
time1=timedelta(days=0,minutes=5)
time2=timedelta(days=0,minutes=10)
time3=timedelta(days=0,minutes=15)

In [35]:
for data2 in d1:
    for c in (data2.index):
        data2.loc[(data2["date"])==(data2["date"][c]+time1),'t1']=data2["avg_travel_time"][c]
        data2.loc[(data2["date"])==(data2["date"][c]+time2),'t2']=data2["avg_travel_time"][c]
        data2.loc[(data2["date"])==(data2["date"][c]+time3),'t3']=data2["avg_travel_time"][c]

In [36]:
for count in range(6):
    for c in range(7):
        for c1 in range(24):
            d1[count].loc[(d1[count]['weekday']==c) & (d1[count]['hour']==c1) & (d1[count]['t1']==0),'t1']=d[count].loc[(d[count]['weekday']==c) & (d[count]['hour']==c1) & (d[count]['t1']!=0)]['t1'].mean()
            d1[count].loc[(d1[count]['weekday']==c) & (d1[count]['hour']==c1) & (d1[count]['t2']==0),'t2']=d[count].loc[(d[count]['weekday']==c) & (d[count]['hour']==c1) & (d[count]['t2']!=0)]['t2'].mean()
            d1[count].loc[(d1[count]['weekday']==c) & (d1[count]['hour']==c1) & (d1[count]['t3']==0),'t3']=d[count].loc[(d[count]['weekday']==c) & (d[count]['hour']==c1) & (d[count]['t3']!=0)]['t3'].mean()

In [37]:
data2=pd.concat(d1,ignore_index=True)

In [38]:
data2['deltat1']=data2['t1']-data2['t2']
data2['deltat2']=data2['t2']-data2['t3']

# Create prediction data

In [39]:
month=[10]*350
starting_time=[]
hours=[]
minute=[]
day=[]
weekday=[]
datetime(2016,4,16,23,55)
for c in range(25,32):
    for c1 in [8,9,10,17,18,19]:
        for c2 in range(0,56,5):
            if (c1==10 or c1==19):
                t=datetime(2016,10,c,c1,0,0)
                weekday.append(t.weekday()+1)
                t=t.strftime('%Y-%m-%d %H:%M:%S')
                starting_time.append(t)
                hours.append(c1)
                minute.append(c2)
                day.append(c)
                break
            else:
                t=datetime(2016,10,c,c1,c2,0)
                weekday.append(t.weekday()+1)
                t=t.strftime('%Y-%m-%d %H:%M:%S')
                starting_time.append(t)
                hours.append(c1)
                minute.append(c2)
                day.append(c)

In [40]:
intersection_id=['A']*350
tollgate_id=[2]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
A2pre=pd.DataFrame(cc)

In [41]:
intersection_id=['A']*350
tollgate_id=[3]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
A3pre=pd.DataFrame(cc)

In [42]:
intersection_id=['B']*350
tollgate_id=[1]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
B1pre=pd.DataFrame(cc)

In [43]:
intersection_id=['B']*350
tollgate_id=[3]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
B3pre=pd.DataFrame(cc)

In [44]:
intersection_id=['C']*350
tollgate_id=[1]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
C1pre=pd.DataFrame(cc)

In [45]:
intersection_id=['C']*350
tollgate_id=[3]*350
travel_time=[0]*350
cc={
    'intersection_id':intersection_id,
    'tollgate_id':tollgate_id,
    'travel_time':travel_time,
    'starting_time':starting_time,
    'weekday':weekday,
    'month':month,
    'day':day,
    'hours':hours,
    'minute':minute
   }
C3pre=pd.DataFrame(cc)

In [46]:
d=[A2,A3,B1,B3,C1,C3]
d1=[A2test,A3test,B1test,B3test,C1test,C3test]
d2=[A2pre,A3pre,B1pre,B3pre,C1pre,C3pre]

In [47]:
time1=timedelta(days=0,minutes=5)
time2=timedelta(days=0,minutes=10)
time3=timedelta(days=0,minutes=15)

In [48]:
for count, pre in enumerate(d2):
    pre.insert(3, "t1", 0.00)
    pre.insert(4, "t2", 0.00)
    pre.insert(5, "t3", 0.00)
    pre.insert(6, "deltat1", 0.00)
    pre.insert(7, "deltat2", 0.00)
    pre['starting_time']=pre['starting_time'].astype(np.datetime64)
    for c in d1[count].index:
        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time1)),'t1']=d1[count]["avg_travel_time"][c]
        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time1)),'t2']=d1[count]["t1"][c]
        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time1)),'t3']=d1[count]["t2"][c]

        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time2)),'t2']=d1[count]["avg_travel_time"][c]
        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time2)),'t3']=d1[count]["t1"][c]

        pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&((pre["starting_time"])==(d1[count]["date"][c]+time3)),'t3']=d1[count]["avg_travel_time"][c]
        
    noise=pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&(pre["t1"]==0)]
    for c in noise.index:
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
                (pre["minute"]==0))&
                (pre["t1"]==0)&
                (pre["weekday"]==noise['weekday'][c]),'t1']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])&(d[count]["weekday"]==noise['weekday'][c])]['t1'].mean()
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
            (pre["minute"]==0))&
            (pre["t1"].isnull())&
            (pre["weekday"]==noise['weekday'][c]),'t1']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])]['t1'].mean()
    noise=pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&(pre["t2"]==0)]
    for c in noise.index:
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
                (pre["minute"]==0))&
                (pre["t2"]==0)&
                (pre["weekday"]==noise['weekday'][c]),'t2']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])&(d[count]["weekday"]==noise['weekday'][c])]['t2'].mean()
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
            (pre["minute"]==0))&
            (pre["t2"].isnull())&
            (pre["weekday"]==noise['weekday'][c]),'t2']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])]['t2'].mean()
    noise=pre.loc[(((pre["hours"]==8)|(pre["hours"]==17))&(pre["minute"]==0))&(pre["t3"]==0)]
    for c in noise.index:
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
                (pre["minute"]==0))&
                (pre["t3"]==0)&
                (pre["weekday"]==noise['weekday'][c]),'t3']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])&(d[count]["weekday"]==noise['weekday'][c])]['t3'].mean()
        pre.loc[(((pre["hours"]==noise['hours'][c]))&
            (pre["minute"]==0))&
            (pre["t3"].isnull())&
            (pre["weekday"]==noise['weekday'][c]),'t3']=d[count].loc[(d[count]["hour"]==noise["hours"][c])&(d[count]["time"]==noise["minute"][c])]['t3'].mean()



In [49]:
pre_data=pd.concat(d2,ignore_index=True)

In [50]:
pre_data['travel_time']=pre_data['travel_time'].astype(float)

In [51]:
pre_data['deltat1']=pre_data['t1']-pre_data['t2']
pre_data['deltat2']=pre_data['t2']-pre_data['t3']

In [52]:
a=[]
for c in pre_data['starting_time']:
    a.append((int(c.hour)*60+int(c.minute))/5)
pre_data['check']=a

In [53]:
pre_data.insert(15,"pressure", 0)
pre_data.insert(16,"sea_pressure", 0)
pre_data.insert(17,"wind_direction", 0)
pre_data.insert(18,"wind_speed", 0)
pre_data.insert(19,"temperature", 0)
pre_data.insert(20,"rel_humidity", 0)
pre_data.insert(21,"precipitation", 0)

In [54]:
weath=pd.read_csv('./weather (table 7)_2.csv')
weath['date']=weath['date'].astype(np.datetime64)
weath['month']=weath['date'].dt.month
weath['day']=weath['date'].dt.day

In [55]:
M=[]
H=[]
day=[]
month=[]
for c in weath["date"]:
    day.append(int(c.strftime("%d")))
    month.append(int(c.strftime("%m")))
weath["month"]=month
weath["day"]=day

In [56]:
for c in range(len(weath)):
    pre_data.loc[((data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'pressure']=weath['pressure'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'sea_pressure']=weath['sea_pressure'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'wind_direction']=weath['wind_direction'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'wind_speed']=weath['wind_speed'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'temperature']=weath['temperature'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'rel_humidity']=weath['rel_humidity'][c]
    pre_data.loc[((pre_data['month']==weath['month'][c])&(pre_data['day']==weath['day'][c]))&
         (((pre_data['hours']-weath['hour'][c])>=0)&((pre_data['hours']-weath['hour'][c])<3)),'precipitation']=weath['precipitation'][c]

In [57]:
pre_data.to_csv('predict_data.csv',index=False)

In [58]:
print ('end data preparation')

end data preparation
